# CityBikes

### Send a request to CityBikes for the city of your choice

In [1]:
import pandas as pd
import json
import requests # This library will be used to call the APIs

In [2]:
# URL of API endpoint
city_bike_networks = "https://api.citybik.es/v2/networks/"

# Returns all available cities covered by CityBikes API
response_networks = requests.get(city_bike_networks)

In [3]:
# Filter for a specific city, in this case we will select 'Montréal, QC, CA: Bixi'
network_id = "bixi-montreal"
city_bike_MTL = city_bike_networks + network_id

# Returns all available bike stations in 'Montréal, QC, CA' using try/excepts considered relevant
try:
    response_MTL = requests.get(city_bike_MTL)
    # If the request is successful
    print("Request successful. Status code:", response_MTL.status_code)
    
# HTTP error for a status codes like 404 (page not found)
except requests.exceptions.HTTPError as http_err:
    print("HTTP error occurred. Error:", str(http_err))

# Server takes too long to respond
except requests.exceptions.Timeout as timeout_err:
    print("Request timed out. Error:", str(timeout_err))
        
except Exception as exc:
    print("Request failed. Error:", str(exc))

Request successful. Status code: 200


In [4]:
# We can convert data into JSON string format with indentation to view its structure
city_bike_view_data = json.dumps(response_MTL.json(), indent=3)

# Print the first 2000 characters of the JSON string
print(city_bike_view_data[:2000])

{
   "network": {
      "company": [
         "Motivate International, Inc.",
         "PBSC Urban Solutions",
         "BIXI Montr\u00e9al"
      ],
      "ebikes": true,
      "gbfs_href": "https://gbfs.velobixi.com/gbfs/gbfs.json",
      "href": "/v2/networks/bixi-montreal",
      "id": "bixi-montreal",
      "location": {
         "city": "Montr\u00e9al, QC",
         "country": "CA",
         "latitude": 45.508693,
         "longitude": -73.553928
      },
      "name": "Bixi",
      "stations": [
         {
            "empty_slots": 6,
            "extra": {
               "ebikes": 0,
               "has_ebikes": true,
               "last_updated": 1695527911,
               "payment": [
                  "creditcard",
                  "key"
               ],
               "payment-terminal": true,
               "renting": 1,
               "returning": 1,
               "slots": 11,
               "uid": "693"
            },
            "free_bikes": 5,
            "id": "

### Parse through response to get the details for the bike stations in that city (latitude, longitude, number of bikes)

In [5]:
# In this case, we will work directly with the JSON response as it is easier to parse using keys and indices
city_bike_data = response_MTL.json()

# For example, we can use the keys and indices to access details about the first station
city_bike_data['network']['stations'][0]

{'empty_slots': 6,
 'extra': {'ebikes': 0,
  'has_ebikes': True,
  'last_updated': 1695527911,
  'payment': ['creditcard', 'key'],
  'payment-terminal': True,
  'renting': 1,
  'returning': 1,
  'slots': 11,
  'uid': '693'},
 'free_bikes': 5,
 'id': '72bfd647b3d2b650546f42319729757d',
 'latitude': 45.617499783128075,
 'longitude': -73.60601127147675,
 'name': 'Cégep Marie-Victorin',
 'timestamp': '2023-09-24T05:56:23.293000Z'}

In [6]:
stations_data = city_bike_data['network']['stations']
stations_details = []

for station in stations_data:
   
    station_details = {
        'station_name': station['name'],
        'city_bike_latitude': station['latitude'],
        'city_bike_longitude': station['longitude'],
        'free_bikes': station['free_bikes'],
        'empty_slots': station['empty_slots'],
        # From the previous code block, we see "has_ebikes = True", but this may not be the case for all stations
        # Use 'get()' method to check if the station has an 'ebike', if not, return 0 (to prevent runtime errors)
        'ebikes': station['extra'].get('ebikes', 0) 
    }
    
    stations_details.append(station_details)

### Put parsed results into a dataframe

In [7]:
all_stations_df = pd.DataFrame(stations_details)
all_stations_df

,station_name,city_bike_latitude,city_bike_longitude,free_bikes,empty_slots,ebikes
0,Cégep Marie-Victorin,45.617500,-73.606011,5,6,0
1,Gare d'autocars de Montréal (Berri / Ontario),45.516926,-73.564257,1,12,0
2,Ateliers municipaux de St-Laurent (Cavendish /...,45.506176,-73.711186,6,12,3
3,Place Rodolphe-Rousseau (Gohier / Édouard-Laurin),45.512994,-73.682498,14,10,13
4,Centre des loisirs (Tassé / Grenet),45.514734,-73.691449,6,8,3
...,...,...,...,...,...,...
793,Queen / Wellington,45.497605,-73.555350,1,42,0
794,Cathcart / McGill College,45.501188,-73.570455,1,26,0
795,de la Montagne / Notre-Dame,45.492913,-73.564688,17,5,0
796,Parc du Pélican (1ere Ave / Masson),45.545188,-73.576443,20,16,8


In [8]:
# Check for duplicated rows using 'duplicated()' method which returns boolean Series denoting duplicate rows
print(all_stations_df.duplicated().value_counts())

# Check for missing values in each column
print(f"\n{all_stations_df.isna().sum()}")

# Examine the data types of each column to see if they make sense
print(f"\n{all_stations_df.dtypes}")

False    798
Name: count, dtype: int64

station_name           0
city_bike_latitude     0
city_bike_longitude    0
free_bikes             0
empty_slots            0
ebikes                 0
dtype: int64

station_name            object
city_bike_latitude     float64
city_bike_longitude    float64
free_bikes               int64
empty_slots              int64
ebikes                   int64
dtype: object


In [9]:
# Save DataFrame to a CSV file without row index value
all_stations_df.to_csv("../data/montreal_bike_stations.csv", index = False)